- Обучите CNN (самописная) на CIFAR-100.

D. Определите размер карты признаков на выходе, если на вход слоя свертки пришел тензор 1х128х100х100, а сверточный слой имеет фильтр размера 3, шаг = 1 и использует заполнение 0  
1 128 98 98  

E. Определите размер карты признаков на выходе, если на вход слоя пуллинга пришел тензор 1х128х100х100, а сверточный слой имеет фильтр размера 2, шаг = 2 и использует заполнение 0  
1 128 50 50 (шаг 2 уменьшает в 2 раза размер)

In [1]:
import torch
import numpy as np
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch import nn
from torch import optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [16]:
!pip install torchsummary

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Прогрузка данных

In [9]:
train_dataset = torchvision.datasets.CIFAR100(root='data/',
                                             train=True,
                                             transform=transforms.ToTensor(),
                                             download=True)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transforms.ToTensor())


Files already downloaded and verified
Files already downloaded and verified


In [10]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [11]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=500,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=500,
                                         shuffle=True)

## Модель

In [12]:
class Net_CCN_5(nn.Module):

    def __init__(self):
        super(Net_CCN_5, self).__init__()

        self.dp1 = nn.Dropout(0.2)
        self.bn1 = torch.nn.BatchNorm2d(3)
        self.conv1 = torch.nn.Conv2d(3, 30, 3)#32

        self.bn2 = torch.nn.BatchNorm2d(30)
        self.conv2 = torch.nn.Conv2d(30, 64, 3) #32
        self.dp2 = nn.Dropout(0.3)

        self.bn3= torch.nn.BatchNorm2d(64)
        self.conv3 = torch.nn.Conv2d(64, 128, 3) #32 вход
        self.dp3 = nn.Dropout(0.3)
        #polling запулингировали 16

        self.bn4 = torch.nn.BatchNorm2d(128)
        self.conv4 = torch.nn.Conv2d(128, 128, 3) #16 в 14
        self.dp4 = nn.Dropout(0.2)

        self.bn5= torch.nn.BatchNorm2d(128)
        self.conv5 = torch.nn.Conv2d(128, 256, 3) #14 в 12
        self.dp5 = nn.Dropout(0.2)

        #polling #из 12 в 6

        self.bn6 = torch.nn.BatchNorm2d(256)
        self.conv6 = torch.nn.Conv2d(256, 256, 3) #из 6 в 4

        self.bn7 = torch.nn.BatchNorm2d(256)
        self.conv7 = torch.nn.Conv2d(256, 512, 3) # 2
        #self.fl = torch.nn.Flatten(2)

        self.bn8 = torch.nn.BatchNorm2d(512)

        self.fc1 = torch.nn.Linear(2048, 1024) #256 на 3 на 3
        self.dp_fc1 = nn.Dropout(0.3)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.dp_fc2 = nn.Dropout(0.3)
        self.out = torch.nn.Linear(512, 100)

    def forward(self, x):

        x = self.dp1(x)
        x = self.bn1(x) # 32 на 32
        x = torch.nn.functional.pad(x, (1, 1, 1, 1), mode='reflect')
        x = self.conv1(x) # 32 на 32 падинг 1
        x = F.relu(x)

        x = self.bn2(x)
        x = torch.nn.functional.pad(x, (1, 1, 1, 1), mode='reflect')
        x = self.conv2(x) #30 на 30
        x = self.dp2(x)
        x = F.relu(x)

        x = self.bn3(x)
        x = torch.nn.functional.pad(x, (1, 1, 1, 1), mode='reflect')
        x = self.conv3(x) #28 на 28
        x = self.dp3(x)
        x = F.relu(x)

        x = F.avg_pool2d(x,2,stride=2) #14 на 14

        x = self.bn4(x) #14 на 14
        x = self.conv4(x) #12 на 12
        x = self.dp4(x)
        x = F.relu(x)

        x = self.bn5(x)
        x = self.conv5(x) #
        x = self.dp5(x)
        x = F.relu(x)

        x = F.avg_pool2d(x,2,stride=2)

        x = self.bn6(x)
        x = self.conv6(x)
        x = F.relu(x)

        x = self.bn7(x)
        x = self.conv7(x)
        x = F.relu(x)
        #x = self.fl(x)
        x = self.bn8(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dp_fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = self.dp_fc2(x)
        x = F.relu(x)

        return self.out(x)

net = Net_CCN_5().to(device)
#print(net)

from torchsummary import summary

summary(net, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Dropout-1            [-1, 3, 32, 32]               0
       BatchNorm2d-2            [-1, 3, 32, 32]               6
            Conv2d-3           [-1, 30, 32, 32]             840
       BatchNorm2d-4           [-1, 30, 32, 32]              60
            Conv2d-5           [-1, 64, 32, 32]          17,344
           Dropout-6           [-1, 64, 32, 32]               0
       BatchNorm2d-7           [-1, 64, 32, 32]             128
            Conv2d-8          [-1, 128, 32, 32]          73,856
           Dropout-9          [-1, 128, 32, 32]               0
      BatchNorm2d-10          [-1, 128, 16, 16]             256
           Conv2d-11          [-1, 128, 14, 14]         147,584
          Dropout-12          [-1, 128, 14, 14]               0
      BatchNorm2d-13          [-1, 128, 14, 14]             256
           Conv2d-14          [-1, 256,

In [36]:
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.007)
criterion = nn.CrossEntropyLoss()

In [37]:
num_epochs = 100
net.train()

for epoch in range(num_epochs):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)  #to(device) .to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()

        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()

            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.4f}. ' \
                  f'Acc: {(running_right / running_items)*100:.4f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):

                test_outputs = net(data[0].to(device)) #.to(device)
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum() #to(device)

            print(f'Test acc: {(test_running_right / test_running_total)*100:.4f}')

        net.train()

print('Training is finished!')

Epoch [1/100]. Step [1/100]. Loss: 0.0092. Acc: 0.8000. Test acc: 1.0000
Epoch [2/100]. Step [1/100]. Loss: 0.0092. Acc: 2.6000. Test acc: 2.1000
Epoch [3/100]. Step [1/100]. Loss: 0.0090. Acc: 2.0000. Test acc: 2.6900
Epoch [4/100]. Step [1/100]. Loss: 0.0089. Acc: 2.4000. Test acc: 3.2900
Epoch [5/100]. Step [1/100]. Loss: 0.0089. Acc: 3.0000. Test acc: 3.2900
Epoch [6/100]. Step [1/100]. Loss: 0.0086. Acc: 4.2000. Test acc: 4.4100
Epoch [7/100]. Step [1/100]. Loss: 0.0084. Acc: 5.2000. Test acc: 3.5600
Epoch [8/100]. Step [1/100]. Loss: 0.0086. Acc: 4.0000. Test acc: 4.1900
Epoch [9/100]. Step [1/100]. Loss: 0.0089. Acc: 4.6000. Test acc: 4.1300
Epoch [10/100]. Step [1/100]. Loss: 0.0084. Acc: 7.0000. Test acc: 5.2100
Epoch [11/100]. Step [1/100]. Loss: 0.0084. Acc: 3.4000. Test acc: 5.2200
Epoch [12/100]. Step [1/100]. Loss: 0.0085. Acc: 6.8000. Test acc: 4.8800
Epoch [13/100]. Step [1/100]. Loss: 0.0085. Acc: 4.6000. Test acc: 5.3600
Epoch [14/100]. Step [1/100]. Loss: 0.0081. Acc

In [38]:
num_epochs = 40
net.train()

for epoch in range(num_epochs):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)  #to(device) .to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()

        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()

            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.4f}. ' \
                  f'Acc: {(running_right / running_items)*100:.4f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):

                test_outputs = net(data[0].to(device)) #.to(device)
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum() #to(device)

            print(f'Test acc: {(test_running_right / test_running_total)*100:.4f}')

        net.train()

print('Training is finished!')

Epoch [1/40]. Step [1/100]. Loss: 0.0034. Acc: 53.0000. Test acc: 35.6400
Epoch [2/40]. Step [1/100]. Loss: 0.0034. Acc: 56.0000. Test acc: 36.1200
Epoch [3/40]. Step [1/100]. Loss: 0.0037. Acc: 51.0000. Test acc: 33.1200
Epoch [4/40]. Step [1/100]. Loss: 0.0032. Acc: 54.2000. Test acc: 35.0900
Epoch [5/40]. Step [1/100]. Loss: 0.0031. Acc: 56.6000. Test acc: 35.7400
Epoch [6/40]. Step [1/100]. Loss: 0.0031. Acc: 56.8000. Test acc: 35.1300
Epoch [7/40]. Step [1/100]. Loss: 0.0033. Acc: 53.8000. Test acc: 36.1800
Epoch [8/40]. Step [1/100]. Loss: 0.0031. Acc: 54.6000. Test acc: 35.3200
Epoch [9/40]. Step [1/100]. Loss: 0.0031. Acc: 57.2000. Test acc: 35.3500
Epoch [10/40]. Step [1/100]. Loss: 0.0031. Acc: 56.0000. Test acc: 36.5500
Epoch [11/40]. Step [1/100]. Loss: 0.0031. Acc: 58.2000. Test acc: 35.4200
Epoch [12/40]. Step [1/100]. Loss: 0.0031. Acc: 58.8000. Test acc: 35.0500
Epoch [13/40]. Step [1/100]. Loss: 0.0033. Acc: 56.8000. Test acc: 35.4500
Epoch [14/40]. Step [1/100]. Loss:

In [39]:
num_epochs = 60
net.train()

for epoch in range(num_epochs):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)  #to(device) .to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()

        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()

            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.4f}. ' \
                  f'Acc: {(running_right / running_items)*100:.4f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):

                test_outputs = net(data[0].to(device)) #.to(device)
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum() #to(device)

            print(f'Test acc: {(test_running_right / test_running_total)*100:.4f}')

        net.train()

print('Training is finished!')

Epoch [1/60]. Step [1/100]. Loss: 0.0026. Acc: 62.2000. Test acc: 38.0300
Epoch [2/60]. Step [1/100]. Loss: 0.0023. Acc: 70.2000. Test acc: 37.2500
Epoch [3/60]. Step [1/100]. Loss: 0.0025. Acc: 64.6000. Test acc: 35.7800
Epoch [4/60]. Step [1/100]. Loss: 0.0026. Acc: 65.8000. Test acc: 34.1400
Epoch [5/60]. Step [1/100]. Loss: 0.0028. Acc: 62.2000. Test acc: 37.5400
Epoch [6/60]. Step [1/100]. Loss: 0.0027. Acc: 63.0000. Test acc: 37.3500
Epoch [7/60]. Step [1/100]. Loss: 0.0029. Acc: 58.0000. Test acc: 33.8100
Epoch [8/60]. Step [1/100]. Loss: 0.0024. Acc: 68.4000. Test acc: 36.8200
Epoch [9/60]. Step [1/100]. Loss: 0.0026. Acc: 65.8000. Test acc: 36.9100
Epoch [10/60]. Step [1/100]. Loss: 0.0026. Acc: 65.8000. Test acc: 37.7100
Epoch [11/60]. Step [1/100]. Loss: 0.0025. Acc: 66.6000. Test acc: 37.3900
Epoch [12/60]. Step [1/100]. Loss: 0.0021. Acc: 67.8000. Test acc: 39.5100
Epoch [13/60]. Step [1/100]. Loss: 0.0024. Acc: 66.0000. Test acc: 38.7800
Epoch [14/60]. Step [1/100]. Loss:

In [40]:
num_epochs = 40
net.train()

for epoch in range(num_epochs):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)  #to(device) .to(device)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()

        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()

            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.4f}. ' \
                  f'Acc: {(running_right / running_items)*100:.4f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):

                test_outputs = net(data[0].to(device)) #.to(device)
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum() #to(device)

            print(f'Test acc: {(test_running_right / test_running_total)*100:.4f}')

        net.train()

print('Training is finished!')

Epoch [1/40]. Step [1/100]. Loss: 0.0017. Acc: 77.6000. Test acc: 38.7800
Epoch [2/40]. Step [1/100]. Loss: 0.0019. Acc: 72.6000. Test acc: 37.1000
Epoch [3/40]. Step [1/100]. Loss: 0.0017. Acc: 77.6000. Test acc: 38.5000
Epoch [4/40]. Step [1/100]. Loss: 0.0015. Acc: 79.0000. Test acc: 38.0600
Epoch [5/40]. Step [1/100]. Loss: 0.0017. Acc: 77.4000. Test acc: 39.1200
Epoch [6/40]. Step [1/100]. Loss: 0.0017. Acc: 75.6000. Test acc: 39.0400
Epoch [7/40]. Step [1/100]. Loss: 0.0015. Acc: 78.6000. Test acc: 38.9700
Epoch [8/40]. Step [1/100]. Loss: 0.0016. Acc: 75.6000. Test acc: 37.1300
Epoch [9/40]. Step [1/100]. Loss: 0.0017. Acc: 78.0000. Test acc: 38.5300
Epoch [10/40]. Step [1/100]. Loss: 0.0018. Acc: 75.2000. Test acc: 38.8100
Epoch [11/40]. Step [1/100]. Loss: 0.0021. Acc: 73.4000. Test acc: 38.1500
Epoch [12/40]. Step [1/100]. Loss: 0.0016. Acc: 78.4000. Test acc: 39.5900
Epoch [13/40]. Step [1/100]. Loss: 0.0018. Acc: 74.0000. Test acc: 38.3000
Epoch [14/40]. Step [1/100]. Loss:

In [41]:
PATH_WEIGHTS = './cnn5_weights.pth'
torch.save(net.state_dict(), PATH_WEIGHTS)

In [42]:
print("Model state_dict: ")
for param in net.state_dict():
    print(param, "\t", net.state_dict()[param].size())

Model state_dict: 
bn1.weight 	 torch.Size([3])
bn1.bias 	 torch.Size([3])
bn1.running_mean 	 torch.Size([3])
bn1.running_var 	 torch.Size([3])
bn1.num_batches_tracked 	 torch.Size([])
conv1.weight 	 torch.Size([30, 3, 3, 3])
conv1.bias 	 torch.Size([30])
bn2.weight 	 torch.Size([30])
bn2.bias 	 torch.Size([30])
bn2.running_mean 	 torch.Size([30])
bn2.running_var 	 torch.Size([30])
bn2.num_batches_tracked 	 torch.Size([])
conv2.weight 	 torch.Size([64, 30, 3, 3])
conv2.bias 	 torch.Size([64])
bn3.weight 	 torch.Size([64])
bn3.bias 	 torch.Size([64])
bn3.running_mean 	 torch.Size([64])
bn3.running_var 	 torch.Size([64])
bn3.num_batches_tracked 	 torch.Size([])
conv3.weight 	 torch.Size([128, 64, 3, 3])
conv3.bias 	 torch.Size([128])
bn4.weight 	 torch.Size([128])
bn4.bias 	 torch.Size([128])
bn4.running_mean 	 torch.Size([128])
bn4.running_var 	 torch.Size([128])
bn4.num_batches_tracked 	 torch.Size([])
conv4.weight 	 torch.Size([128, 128, 3, 3])
conv4.bias 	 torch.Size([128])
bn5.weigh

In [13]:
net = Net_CCN_5().to(device)
PATH_WEIGHTS = './cnn5_weights.pth'
net.load_state_dict(torch.load(PATH_WEIGHTS))
net


Net_CCN_5(
  (dp1): Dropout(p=0.2, inplace=False)
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(30, 64, kernel_size=(3, 3), stride=(1, 1))
  (dp2): Dropout(p=0.3, inplace=False)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (dp3): Dropout(p=0.3, inplace=False)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (dp4): Dropout(p=0.2, inplace=False)
  (bn5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (dp5): Dropout(p=0.2, inplace=False)
  (bn6): BatchNorm2d(256, eps=1e-05, mo

In [15]:
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('The final Test Accuracy of the model: {} %'.format((correct / total) * 100))

The final Test Accuracy of the model: 38.080000000000005 %


* Созданая сеть неплохо с падингам с модом reflection на начальных этапах довольно неплохо обучается

* Однако на 40 эпохе начинается переобучение. Для его устранения дополнительно необходимо подобрать скорость обучения и % Dropout в самой сети так же можно добавить l2 регуляризацию в ходе обучения